In [1]:
from dataset_tf import VoxDataset
#import torch
#from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm as tqdm
from joblib import Parallel, delayed

In [2]:
dataset = VoxDataset()

In [ ]:
dataset[0][1].shape

2024-03-16 21:36:36.398729: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-03-16 21:36:36.398751: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-16 21:36:36.398755: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-16 21:36:36.398780: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-16 21:36:36.398797: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


TensorShape([15069])

In [ ]:
np.max(dataset[0][1])

270.77847

In [ ]:
max = 0

for i in tqdm(dataset):
    curr_max = np.max(i[1])
    if curr_max > max:
        max = curr_max

100%|██████████| 107640/107640 [01:58<00:00, 908.41it/s]


In [ ]:
max

279.79984

In [ ]:
max = 0


for i in tqdm(dataset):
    curr_max = np.max(i[0])
    if curr_max > max:
        max = curr_max

100%|██████████| 107640/107640 [02:20<00:00, 763.49it/s]


In [ ]:
max

226.92715

In [ ]:
baseline = dataset[0][1]
dist = []
for index, i in enumerate(dataset):
    mae = np.mean(np.abs(i[1] - baseline))
    dist.append(mae)
    print(f"{index}/{len(dataset)} ------ MAE: {mae}", end = "\r")

In [ ]:
np.mean(dist)

0.0061566182

In [ ]:
ids = dataset.get_ids()

107640


In [ ]:
train_indices = ids[:int(len(ids) * 0.8)]
test_indices = ids[int(len(ids) * 0.8):]

In [ ]:
train = VoxDataset(ids=train_indices)
test = VoxDataset(ids=test_indices)

In [ ]:
train_dist = []

for i in tqdm(train):
    train_dist.append(i[0])

100%|██████████| 86100/86100 [01:21<00:00, 1057.56it/s]


In [ ]:
test_dist = []

for i in tqdm(test):
    test_dist.append(i[1])

100%|██████████| 21540/21540 [00:47<00:00, 451.10it/s]


In [ ]:
diff = []
baseline = train_dist[0]

for index, i in enumerate(train_dist):
    mae = np.mean(i-baseline)
    print(f"{index}/{len(train_dist)} ------ MAE: {mae}", end = "\r")

KeyboardInterrupt: 

In [ ]:
y_pred_1 = [i[1] for i in tqdm(dataset)]

  0%|          | 0/107640 [00:00<?, ?it/s]

2024-03-20 23:49:28.889566: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-03-20 23:49:28.889595: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-20 23:49:28.889604: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-20 23:49:28.889641: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-20 23:49:28.889660: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
100%|██████████| 107640/107640 [01:52<00:00, 955.53it/s] 


In [ ]:
y_pred_1[0]

<tf.Tensor: shape=(15069,), dtype=float32, numpy=
array([0.6167108 , 0.48118067, 0.40816686, ..., 0.4403067 , 0.53916496,
       0.51728684], dtype=float32)>

In [ ]:
y_pred = []

for index, i in enumerate(y_pred_1):
    if index % 20 == 0:
        y_pred.append(i.numpy().reshape(5023, 3))

In [ ]:
len(y_pred)

5382

In [ ]:
y_pred[0]

array([[0.6167108 , 0.48118067, 0.40816686],
       [0.6230832 , 0.47950226, 0.41069612],
       [0.62386197, 0.48211312, 0.4113392 ],
       ...,
       [0.43239596, 0.53759164, 0.520634  ],
       [0.43618873, 0.53834605, 0.51856166],
       [0.4403067 , 0.53916496, 0.51728684]], dtype=float32)

In [ ]:
import open3d as o3d
import numpy as np

In [ ]:
for index, i in enumerate(tqdm(y_pred)):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(i)
    # o3d.visualization.draw_geometries([pcd])
    o3d.io.write_point_cloud(f"./Train/data_{index}.ply", pcd)

100%|██████████| 5382/5382 [00:18<00:00, 296.82it/s]


In [4]:
len(dataset.get_ids())

107640


5382

In [14]:
dist = []
for index, i in enumerate(tqdm(dataset)):
    dist.append(i[0].numpy())

100%|██████████| 107640/107640 [03:57<00:00, 454.00it/s]


In [15]:
seen_arrays = set()

# List to store duplicate arrays
duplicate_arrays = []

# Iterate through the list of arrays
for arr in tqdm(dist):
    # Convert the array to a tuple to make it hashable (and add to set)
    arr_tuple = tuple(arr)
    
    # Check if the array is already in the set (indicating a duplicate)
    if arr_tuple in seen_arrays:
        duplicate_arrays.append(arr)
    else:
        seen_arrays.add(arr_tuple)

# Check if duplicates were found
if duplicate_arrays:
    print("Duplicates found in the list:")
    print(len(duplicate_arrays))
    # for arr in duplicate_arrays:
    #     print(arr)
else:
    print("No duplicates found in the list.")

100%|██████████| 107640/107640 [00:03<00:00, 32582.96it/s]

Duplicates found in the list:
59


In [7]:
len(duplicate_arrays)

102258

In [16]:
for i in dist:
    print(i)


-3.8883636e-07
-1.6849575e-06
-1.166509e-06
-1.036897e-06
1.8145696e-06
-2.073794e-06
-9.072848e-07
5.184485e-07
0.0
-7.776727e-07
-1.9441818e-06
-2.203406e-06
-1.2313151e-06
1.6849575e-06
-1.2961212e-07
7.776727e-07
-8.4247876e-07
-5.832545e-07
6.480606e-08
-4.536424e-07
2.203406e-06
4.536424e-07
3.8883636e-07
9.072848e-07
1.166509e-06
-5.184485e-07
1.7497636e-06
1.2961211e-06
-1.3609273e-06
-1.4257333e-06
1.9441818e-06
1.6201515e-06
2.268212e-06
-1.8145696e-06
1.5553454e-06
1.4257333e-06
2.5274362e-06
-1.101703e-06
-2.4626302e-06
-6.480606e-08
-1.9441818e-07
2.1386e-06
5.832545e-07
-2.333018e-06
-2.5274362e-06
-1.6201515e-06
2.8514667e-06
2.5922424e-07
6.4806056e-07
1.2313151e-06
1.2961212e-07
2.073794e-06
8.4247876e-07
1.9441818e-07
1.036897e-06
2.333018e-06
-1.2961211e-06
3.1106908e-06
-2.5922423e-06
1.8793758e-06
-6.4806056e-07
-1.4905394e-06
2.4626302e-06
-2.3978241e-06
-7.1286667e-07
1.3609273e-06
2.7218546e-06
-1.5553454e-06
-9.720909e-07
2.3978241e-06
1.4905394e-06
2.0089878e-